# AWS Lambda

## コンテナ読み込みを支える技術


### 重複削除

- 多くのイメージは同じ内容を含む：Lambda利用者のコンテナイメージの約75%は、ユニークなバイトが5%未満
- 重複を削除して効率的にキャッシュを作る
- 単一のフラット ファイル システムに展開し、そのファイル システムを 512KiB のチャンクに分割する。次に、チャンクをハッシュして一意のコンテンツを識別し、キャッシュ レイヤーに同じデータのコピーが多すぎるのを回避

:::{admonition} Brooker et al. (2023). On-demand Container Loading in AWS Lambda
:class: info

Brooker, M., Danilov, M., Greenwood, C., & Piwonka, P. (2023). On-demand Container Loading in AWS Lambda. In 2023 USENIX Annual Technical Conference (USENIX ATC 23) (pp. 315-328).

- [ATC '23](https://www.usenix.org/conference/atc23/presentation/brooker) (paper & video)
- [arxiv.org/pdf/2305.13162](https://arxiv.org/pdf/2305.13162) (paper)
- [Container Loading in AWS Lambda - Marc's Blog](https://brooker.co.za/blog/2023/05/23/snapshot-loading.html) (Brookerのブログでの解説)
:::

:::{admonition} Deep dive into AWS Lambda security: Function isolation (re:Invent 2021)
:class: info

- video：https://www.youtube.com/watch?v=FTwsMYXWGB0
- Classmethodの解説ブログ：[[セッションレポート]Lambdaのコンテナイメージ管理の裏側に詳しくなれるセッション SVS404 Deep dive into AWS Lambda security: Function isolation #reinvent | DevelopersIO](https://dev.classmethod.jp/articles/reinvent2020-svs404/)

:::

## コンテナイメージの最適化

### 公式ベースイメージは見かけ以上に高速に読み込まれる

個人的には経験上これは感じていたが、実際にAWSのブログにも書かれている

[Optimizing Lambda functions packaged as container images | AWS Compute Blog](https://aws.amazon.com/jp/blogs/compute/optimizing-lambda-functions-packaged-as-container-images/)

の「Use AWS-provided base images」の部分には以下のようなことが書かれている。

> たとえば、AWS が提供する Go ランタイム `public.ecr.aws/lambda/go:1` のベースイメージは 670 MB あり、alpine:latest はわずか 5.58 MB です。ただし、AWS が提供するベースイメージを使用すると、3 つの利点があります。
>
> 1. キャッシュにより高速：AWS が提供するベースイメージは、Lambda サービスによってプロアクティブにキャッシュされます。これは、ベース イメージが近くの別の上流キャッシュにあるか、すでにワーカー インスタンス キャッシュに存在することを意味します。はるかに大きいにもかかわらず、キャッシュされない可能性があるサードパーティの基本イメージと比較すると、展開時間は依然として短い可能性があります。
>
> 2. 安定：AWS が提供するベースイメージは安定しています。基本イメージはコンテナー イメージの最下層にあるため、変更を行うには他のすべての層を再構築して再デプロイする必要があります。基本イメージへの変更が少ないということは、再構築と再デプロイの回数が減り、構築コストを削減できることを意味します。
>
> 3. 様々なソフトウェアが同梱されたOS：AWS が提供するベースイメージは Amazon Linux および Amazon Linux 2 上に構築されています。選択したランタイムに応じて、関数に必要な多数のユーティリティとライブラリがすでに含まれている場合があります。これは、後でそれらを追加する必要がないことを意味し、コストの増加につながるビルド手順の増加を引き起こす可能性がある追加のレイヤーを作成する手間を省きます。
